In [1]:
import pprint
from ollama import chat
import json
from os import listdir
from os.path import join
import re

In [2]:
# def compile_SYS_PROMT():
# 	return """
# 	You are a social innovator, helping me write a glossary on tools, methods, tactics, principles and skills 
#     that are commonly used in social and public innovation work.

#     I will share with you a COMPILATION OF NOTES in markdown format that you will summarize the content for a single entry in the glossary.
#     You return only the summary in a structured markdown format.
# 	"""

In [3]:
def compile_SYS_PROMT():
	return """
	You are a social innovator, helping me write a glossary on tools, methods, tactics, principles and skills 
    that are commonly used in public and social innovation work. The audience of this glossary are public officials
    and civil servants who have some understanding of public and social innovation, but are not experts.

    I will share with you a COMPILATION OF NOTES in markdown format that you will summarize the content for a single entry in the glossary.
    You return only the summary in a structured markdown format, using the structure below. Do not add an introductory sentence to your response.

    ## Overview
    ## Purpose
    ## How it works
    ## Requirements
    ## Reflections
    ## Related methods (if any)
    ## License
    ## Examples
	"""

In [4]:
def summarize_search_result(**kwargs):
	prompt = kwargs.get('prompt')
	md = kwargs.get('md')
	stream_output = kwargs.get('stream_output', False)
	model = kwargs.get('model', 'llama3.1:8b')

	if prompt is None:
		raise Exception('missing prompt')
	if md is None:
		raise Exception('missing json input')

	stream: ChatResponse = chat(
		model=model, 
		options={
			'seed': 42,
			'temperature': 0.2,
			'num_ctx': 10000,
		},
		messages = [
			{
				'role': 'system',
				'content': compile_SYS_PROMT(),
			},
			{
				'role': 'user',
				'content': f"""
					COMPILATION OF NOTES: {md}
					USER PROMPT: {prompt}
				"""
			},
		],
		stream=stream_output,
	)

	full_response = ''

	if stream_output == True:
		print('[AGENT] SUMMARY:')
		for chunk in stream:
			print(chunk['message']['content'], end='', flush=True)
			full_response += chunk['message']['content']
		print('\n')
	else:
		full_response = stream['message']['content']

	return full_response

In [5]:
%run load_docx.ipynb

In [6]:
directory = f'../elements/__pages__/'
files = [f for f in listdir(directory) if f != '.DS_Store']

In [7]:
def getTypes (md):
    types = re.findall(r'(?<=\[\[type:)[\w:.\-\/\d\s…&\(\)\",]*(?=\]\])', str(md))
    types = [t[:-1] for t in types if t[-1] == 's']
    return types

In [8]:
def getElementName (md):
    # print(md.decode('unicode_escape'))
    # print(md)
    element = re.findall(r'(?<=\n# )[\w\'"’”‘“\(\)\[\]_\-\s]+(?=\n)', str(md), flags=re.MULTILINE)
    element = [e.strip() for e in element]
    return element[0]

In [10]:
for f in files[:10]:
    # f = 'Accessibility of knowledge.md'
    md = readFile(join(directory, f))
    element = getElementName(md)
    print(element)
    element_type = ' or '.join(getTypes(md))
    print(element_type)
    prompt = f'I need to write an entry on {element} as a {element_type} for a social innovation glossary'
    # Summarize
    content = summarize_search_result(prompt=prompt, md=md, stream_output=False)
    # Export summary
    output_file = open(join(directory, f), 'w')
    output_file.write(content)
    output_file.close()

Nesta’s DIY Toolkit
tool
Leadership
skill
EGAP Pre-Analysis Plans
tool
Survey 123
tool
Collaboration over competition
principle


KeyboardInterrupt: 